# Example YOLOv2 detector

This notebook is the same as `example_yolo` except that it uses a `YoloGenerator` object for data augmentation. It allows to make transformation such as zoom, flips, rotations, ... and modify boxes accordingly (which is currently not supported by the dataset pipeline). 

However this makes the training 4 times slower so I suggest you to use it only if your dataset is really small. Otherwise, prefer to use the other notebook forfaster training (where there is also augmentation but only on colors / luminosity / contrasts and not on object position). 

## Imports + model initialization

In [1]:
import pandas as pd
import tensorflow as tf

from utils import plot, plot_multiple, set_display_options, limit_gpu_memory
from datasets import get_dataset, prepare_dataset, test_dataset_time
from models.detection import YOLO
from custom_train_objects import YoloGenerator

set_display_options()
#limit_gpu_memory()

input_size = 416
model_name = 'yolo_test'
print("Tensorflow version : {}".format(tf.__version__))

Tensorflow version : 2.5.0


In [ ]:
config = {
    'input_size' : input_size,
    'backend'    : 'full_yolo',
    'nb_class'   : 5,
    'max_box_per_image' : 50,
    'labels'     : ['face']
}
model = YOLO(nom = model_name, ** config)

print(model)

## Model instanciation + dataset loading

In [ ]:
model = YOLO(nom = model_name)

model.compile(
    optimizer = 'sgd',
    optimizer_config = {
        'lr' : {'name' : 'WarmupScheduler', 'maxval' : 1e-3,'minval' : 1e-4, 'factor' : 1024, 'warmup_steps' : 1024}
    },
    loss_config = {'warmup_epochs' : 3}
)

print(model)

In [ ]:
dataset_name = 'wider'
dataset = get_dataset(
    dataset_name, max_box_per_image = model.max_box_per_image, box_as_dict = True
)

train, valid = dataset['train'], dataset['valid']
print('Train size : {} - Valid size : {}'.format(len(train), len(valid)))

In [ ]:
def norm(img):
    return img / 255.

config = {
    'IMAGE_W' : input_size,
    'IMAGE_H' : input_size,
    'LABELS'  : model.labels,
    'BOX'     : model.nb_box,
    'ANCHORS' : model.anchors,
    'GRID_W'  : model.grid_w,
    'GRID_H'  : model.grid_h,
    'TRUE_BOX_BUFFER' : model.max_box_per_image
}
train = YoloGenerator(train.sample(2048, random_state = 0), config, shuffle = True, norm = norm)
valid = YoloGenerator(valid.sample(1024, random_state = 0), config, jitter = False, shuffle = False, norm = norm)

## Training + history analysis

Note that augmentation is already done by the generator so you can remove default augmentation (`augment_prct = 0.`). 

In [ ]:
epochs = 30
batch_size = 16

train_size = None
valid_size = None

model.train(
    train, validation_data = valid, train_times = 8,
    train_size = train_size, valid_size = valid_size,
    epochs = epochs, batch_size = batch_size,
    augment_prct = 0., shuffle_size = 0, cache = False,
    encode_fn = None, augment_fn = None
)

In [ ]:
print(model.history)
model.plot_history(ylim = (0, 1))

## Prediction

In [ ]:
_ = model.predict(pd.DataFrame(train.images[:5]), save = False, debug = True, verbose = 3, obj_threshold = 0.2)

In [ ]:
_ = model.predict('lena.jpg', save = False, debug = True, verbose = 1)

## Streaming

In [ ]:
model.stream(max_time = 15)

## Test dataset

In [ ]:
config = model.get_dataset_config(
    is_validation = False, batch_size = 16, encode_fn = None, augment_fn = None, shuffle_size = 0)

ds_train = prepare_dataset(train, ** config, debug = True)
test_dataset_time(ds_train)

In [ ]:
import numpy as np
from utils.image import *

config = model.get_dataset_config(is_validation = True, batch_size = 1, prefetch = False)

ds_train = prepare_dataset(train.iloc[:5], ** config, debug = True)

for _, data in train.iloc[:5].iterrows():
    y_true, true_boxes = model.get_output(data)
    image = model.get_input(data)
    image = load_image(data['filename'])
    boxes = [list(b) + [1, 0] for b in data['box'].astype(np.int32) if b[0] > 0]
    print(np.array(boxes) / data['width'])
    print(512 / (data['width'] * 13))
    print(model.grid_w)
    boxes = model.decode_output(y_true)
    print(boxes)
    plot(model.draw_prediction(image, boxes, labels = ['face', 'face']))
